In [1]:
import pyDOE
import pandas as pd
import numpy as np
import Utils 
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.optimize import Bounds
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from collections import namedtuple
from matplotlib import cm
import cma
from scipy.optimize import minimize
from scipy.optimize import Bounds
import sys

C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


## Helper Functions

In [2]:
ValueRange = namedtuple('ValueRange', ['min', 'max'])

def determinerange(values):
    """Determine the range of values in each dimension"""
    return ValueRange(np.min(values, axis=0), np.max(values, axis=0))


def linearscaletransform(values, *, range_in=None, range_out=ValueRange(0, 1), scale_only=False):
    """Perform a scale transformation of `values`: [range_in] --> [range_out]"""

    if range_in is None:
        range_in = determinerange(values)
    elif not isinstance(range_in, ValueRange):
        range_in = ValueRange(*range_in)

    if not isinstance(range_out, ValueRange):
        range_out = ValueRange(*range_out)

    scale_out = range_out.max - range_out.min
    scale_in = range_in.max - range_in.min

    if scale_only:
        scaled_values = (values / scale_in) * scale_out
    else:
        scaled_values = (values - range_in.min) / scale_in
        scaled_values = (scaled_values * scale_out) + range_out.min

    return scaled_values


''' Plot the Graph of the Function'''
def plot_the_graph(x,y,z,graph_title):
    fig = plt.figure(figsize=(12,5))
    ax = Axes3D(fig)
    surf = ax.plot_trisurf(x,y,z,cmap=cm.jet, linewidth=0.3,alpha=0.9)
    fig.colorbar(surf, shrink=10, aspect=3)
    ax.set_xlabel('X1')
    ax.set_ylabel('X2')
    ax.set_zlabel('Robust(X1,X2)')
    plt.title(graph_title)
    plt.show()
    fig.savefig(graph_title+'.pdf')

''' Branin Test Example  '''
def branin_test_example(X):
    x,z = X
    A = (z) - ((5.1) / (4 * np.square(np.pi))) * (x **2) + (( (5)/(np.pi) ) * (x) ) - (6)
    B = (10 * ((1) - ((1) / (8 * np.pi) )) ) * (np.cos(x))
    return np.square(A) + B + 10


''' Latin HyperCube Sampling Design of Experiment '''
def DOE(n_obs):
    np.random.seed(0)
    lhd = pyDOE.lhs(n=2, samples=n_obs, criterion='m')
    X1, X2 = lhd[:,0], lhd[:,1]
    return X1,X2


'''Robust Regularization based on minimax Principle 5% perturbations'''
def robust_regularization(X):
    x1,x2 = X
    eps = np.linspace(-1.5,1.5,1000)
    return np.max( branin_test_example([x1+eps,x2+eps]) )

''' Robustness based on a composite function of Mean and STd '''
def composite_robustness(X):
    w=1
    x1,x2 = X
    np.random.seed(0)
    eps = np.random.normal(loc=0.0,scale=0.25,size=10000)
    sample_points = branin_test_example([x1+eps,x2+eps])
    sample_mean = np.mean(sample_points)
    variance = np.square(sample_points-sample_mean)
    std = np.sqrt(np.mean(variance))
    std = np.sqrt(np.mean(variance))
    return sample_mean + w * std

''' Generate Training Data using LHD along side the Output of the Robust System'''
def generate_training_data(n_obs):
    X1,X2 = DOE(n_obs)
    X1 = linearscaletransform(X1,range_out=(-5,10))
    X2 = linearscaletransform(X2,range_out=(0,15))
    f_evaluation = branin_test_example([X1,X2])
    f_original = np.zeros(X1.shape[0])
    minimax_original = np.zeros(X1.shape[0])
    composite_original = np.zeros(X1.shape[0])
    for i in range(X1.shape[0]):
        f_original[i] = branin_test_example([X1[i],X2[i]])
        minimax_original[i] = robust_regularization([X1[i],X2[i]])
        composite_original[i] = composite_robustness([X1[i],X2[i]])
    train = pd.DataFrame()
    train['X1'] = pd.Series(X1)
    train['X2'] = pd.Series(X2)
    train['Y = F(X1,X2)'] = pd.Series(f_original)
    train['Y = robust_regularization(X1,X2)'] = pd.Series(minimax_original)
    train['Y = composite(X1,X2)'] = pd.Series(composite_original)
    train.to_csv('Training_Data_Sets\\'+str(n_obs)+'Samples.csv')
    return train

''' Generate Test Data using LHD along side the Output of the Robust System'''
def generate_test_data(n_obs):
    test = pd.DataFrame()
    np.random.seed(0)
    lhd = pyDOE.lhs(n=2, samples=n_obs, criterion='m')
    X1 = linearscaletransform(lhd[:,0],range_out=(-5,10))
    X2 = linearscaletransform(lhd[:,1],range_out=(0,15))
    test ['X1'] = pd.Series(X1)
    test ['X2'] = pd.Series(X2)
    true_minimax = np.zeros(test.shape[0])
    true_composite = np.zeros(test.shape[0])
    for i in range(test.shape[0]):
        true_minimax[i] = robust_regularization(test.iloc[i,:])
        true_composite[i] = composite_robustness(test.iloc[i,:])
    test ['True_Minimax'] = pd.Series(true_minimax)
    test ['True_Composite'] = pd.Series(true_composite)
    test.to_csv('Test_Data_Sets\\'+str(n_obs)+'Samples.csv')
    return test

''' Check if There are same examples in Test and Train Data Sets'''
def check_data_sets(train,test):
    C = train.isin(test)
    C = pd.DataFrame.sum(C,axis=1)
    ids = C.index[C>1]
    print ('Total Number of Input Observations present in Test Data : '+str(ids.shape[0]))

## CMA-ES

In [3]:
opt = {'seed': 0 , 'maxfevals': 1e6  }
rr = cma.fmin(robust_regularization, 2 * [0] , sigma0= 3.75 , options=opt  )
rc = cma.fmin(composite_robustness, 2 * [0] , 3.75 , options=opt  )

(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=121209, Sun Mar 31 11:39:40 2019)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1      6 2.486326968047820e+01 1.0e+00 4.05e+00  4e+00  4e+00 0:00.0
    2     12 3.986238421966755e+01 1.4e+00 4.12e+00  4e+00  4e+00 0:00.0
    3     18 2.424964276273991e+01 1.3e+00 3.04e+00  2e+00  3e+00 0:00.0
  100    600 1.960211009934727e+01 1.4e+02 9.97e-02  7e-03  7e-03 0:00.3
  200   1200 1.960210460313542e+01 4.2e+01 8.09e-01  3e-03  4e-03 0:00.5
  300   1800 1.960210393385017e+01 7.7e+01 3.15e-01  5e-06  2e-05 0:00.7
  349   2094 1.960210393152306e+01 1.3e+04 3.06e-03  1e-08  5e-08 0:00.8
termination on tolfun=1e-11 (Sun Mar 31 11:39:43 2019)
final/bestever f-value = 1.960210e+01 1.712800e+01
incumbent solution: [5.136038171900966, -0.047146796665600954]
std deviation: [1.0413593358059906e-08, 5.1969054893600705e-08]
(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=188263, Sun Mar 31 11:39:43 2019)
Iter

In [4]:
print ('Xmin for RR :'+str(rr[0]))
print ('Fmin for RR :'+str(rr[1]))
print ('Xmin for RC :'+str(rc[0]))
print ('Fmin for RC :'+str(rc[1]))

Xmin for RR :[3.75671208 0.93170633]
Fmin for RR :17.127997820824547
Xmin for RC :[ 3.29204425 -1.01869369]
Fmin for RC :13.699552455279118


## SLSQP

In [5]:
const = Bounds([-5, 0], [10, 15])
min_robust = minimize(robust_regularization,(0,0),method='SLSQP',bounds=const)
min_compo = minimize(composite_robustness,(0,0),method='SLSQP',bounds=const)

In [6]:
min_robust

     fun: 9.375847199621914
     jac: array([9.01394284, 0.45835221])
 message: 'Optimization terminated successfully.'
    nfev: 62
     nit: 11
    njev: 11
  status: 0
 success: True
       x: array([9.4250444 , 2.76045122])

In [7]:
min_compo

     fun: 1.5481468050010196
     jac: array([0.00035837, 0.00019978])
 message: 'Optimization terminated successfully.'
    nfev: 31
     nit: 7
    njev: 7
  status: 0
 success: True
       x: array([3.15461117, 2.28038591])

In [8]:
np.random.seed(0)
lhd = pyDOE.lhs(n=2, samples= 1000, criterion='m')
X1 = linearscaletransform(lhd[:,0],range_out=(-5,10))
X2 = linearscaletransform(lhd[:,1],range_out=(0,15))
Data = pd.DataFrame()
Data['X1'] = pd.Series(X1)
Data['X2'] = pd.Series(X2)
Data.to_csv('Optimization_Test.csv')

## Meta-Modelling

In [9]:
train = pd.read_csv(sys.path[1]+str('\\Training_Data_Sets\\100Samples.csv')).iloc[:,1:]
test = pd.read_csv('Optimization_Test.csv').iloc[:,1:]
model_k_m,pred_k_m = Utils.kriging_minimax(train,test.iloc[:,:2])
model_k_c,pred_k_c = Utils.kriging_composite(train,test.iloc[:,:2])
model_r_m,pred_r_m = Utils.rf_minimax(train,test.iloc[:,:2])
model_r_c,pred_r_c = Utils.rf_composite(train,test.iloc[:,:2])
model_n_m,pred_n_m = Utils.KNN_minimax(train,test.iloc[:,:2])
model_n_c,pred_n_c = Utils.KNN_composite(train,test.iloc[:,:2])
model_s_m,pred_s_m = Utils.SVR_minimax(train,test.iloc[:,:2])
model_s_c,pred_s_c = Utils.SVR_composite(train,test.iloc[:,:2])
model_b_m,pred_b_m = Utils.RBF_minimax(train,test.iloc[:,:2])
model_b_c,pred_b_c = Utils.RBF_composite(train,test.iloc[:,:2])
train ['X1X2'] = pd.Series( np.array(train.X1) * np.array(train.X2) )
train ['X1**2'] = pd.Series ( np.array(train.X1)**2 )
train ['X2**2'] = pd.Series ( np.array(train.X2)**2 )
f_original = train['Y = F(X1,X2)']
robust_original = train['Y = robust_regularization(X1,X2)']
composite_original = train ['Y = composite(X1,X2)']
del train['Y = F(X1,X2)']
del train['Y = robust_regularization(X1,X2)']
del train ['Y = composite(X1,X2)']
train['Y = F(X1,X2)'] = f_original
train['Y = robust_regularization(X1,X2)'] = robust_original
train ['Y = composite(X1,X2)'] = composite_original
test ['X1X2'] =  pd.Series( np.array(test.X1) * np.array(test.X2) )
test ['X1**2'] = pd.Series ( np.array(test.X1)**2 )
test ['X2**2'] = pd.Series ( np.array(test.X2)**2 )
model_m_m,pred_m_m = Utils.elastic_net_minimax(train,test.iloc[:,:5])
model_m_c,pred_m_c = Utils.elastic_net_composite(train,test.iloc[:,:5])
del train['X1X2']
del train['X1**2']
del train['X2**2']
del test['X1X2']
del test['X1**2']
del test['X2**2']

C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1500).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
C:\Users\sefi\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [10]:
print ('RR :  Minimum Using Kriging is : '+str ( robust_regularization(np.array(test.iloc[np.argmin(pred_k_m),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_k_m),:])))
print ('RC :  Minimum Using Kriging is : '+str ( composite_robustness(np.array(test.iloc[np.argmin(pred_k_c),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_k_c),:])))

RR :  Minimum Using Kriging is : 15.837213443979943
At X = :[9.90197055 1.71901921]
RC :  Minimum Using Kriging is : 1.1658489412799193
At X = :[9.36959876 2.26244102]


In [11]:
print ('RR :  Minimum Using SVR  is : '+str ( robust_regularization(np.array(test.iloc[np.argmin(pred_s_m),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_s_m),:])))
print ('RC :  Minimum Using SVR is : '+str ( composite_robustness(np.array(test.iloc[np.argmin(pred_s_c),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_s_c),:])))

RR :  Minimum Using SVR  is : 10.3586683204576
At X = :[9.36959876 2.26244102]
RC :  Minimum Using SVR is : 1.1658489412799193
At X = :[9.36959876 2.26244102]


In [12]:
print ('RR :  Minimum Using KNN is : '+str ( robust_regularization(np.array(test.iloc[np.argmin(pred_n_m),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_n_m),:])))
print ('RC :  Minimum Using KNN is : '+str ( composite_robustness(np.array(test.iloc[np.argmin(pred_n_c),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_n_c),:])))

RR :  Minimum Using KNN is : 11.363310676247753
At X = :[9.63251943 3.02129638]
RC :  Minimum Using KNN is : 1.6271854313821232
At X = :[9.63251943 3.02129638]


In [13]:
print ('RR :  Minimum Using RF is : '+str ( robust_regularization(np.array(test.iloc[np.argmin(pred_r_m),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_r_m),:])))
print ('RC :  Minimum Using RF is : '+str ( composite_robustness(np.array(test.iloc[np.argmin(pred_r_c),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_r_c),:])))

RR :  Minimum Using RF is : 11.363310676247753
At X = :[9.63251943 3.02129638]
RC :  Minimum Using RF is : 1.6271854313821232
At X = :[9.63251943 3.02129638]


In [14]:
print ('RR :  Minimum Using RBFN is : '+str ( robust_regularization(np.array(test.iloc[np.argmin(pred_b_m),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_b_m),:])))
print ('RC :  Minimum Using RBFN is : '+str ( composite_robustness(np.array(test.iloc[np.argmin(pred_b_c),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_b_c),:])))

RR :  Minimum Using RBFN is : 11.389760128479422
At X = :[9.21512265 3.15684197]
RC :  Minimum Using RBFN is : 1.1658489412799193
At X = :[9.36959876 2.26244102]


In [15]:
print ('RR :  Minimum Using ELN is : '+str ( robust_regularization(np.array(test.iloc[np.argmin(pred_m_m),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_m_m),:])))
print ('RC :  Minimum Using ELN is : '+str ( composite_robustness(np.array(test.iloc[np.argmin(pred_m_c),:]))))
print ('At X = :'+str(np.array(test.iloc[np.argmin(pred_m_c),:])))

RR :  Minimum Using ELN is : 19.53933582622021
At X = :[9.68565187 0.08147672]
RC :  Minimum Using ELN is : 8.560527081294302
At X = :[9.68565187 0.08147672]
